In [7]:
from getpass import getpass
import requests
import pandas as pd
import math
from serlo.mysql import db

MAILCHIMP_API_KEY = getpass("MAILCHIMP_API_KEY")
dc = MAILCHIMP_API_KEY.split("-")[1]

mailchimp_api_url = f"https://{dc}.api.mailchimp.com/3.0/lists/a7bb2bbc4f/members"

MAILCHIMP_API_KEY········


In [8]:
# Hier das Datum eintragen, ab dem Subscriptions gesucht werden sollen.

date = "2020-03-01T00:00:00+00:00"

In [16]:
# TODO: Entscheiden, ob die Description hier rausgenommen wird, da es ein anderer Anwendungsfall ist
df = pd.read_sql(
    """
    Select
        user.id as serlo_id,
        user.username,
        user.description,
        user.date,
        sum(events.edits) as edits,
        sum(events.reviews) as reviews,
        sum(events.comments) as comments,
        sum(events.taxonomy) as taxonomy
    From (
        SELECT
            IF(event_log.event_id = 5, 1, 0) as edits,
            IF(event_log.event_id in (6,11), 1, 0) as reviews,
            IF(event_log.event_id in (8,9,14,16), 1, 0) as comments,
            IF(event_log.event_id in (1,2,12,15,17), 1, 0) as taxonomy,
            event_log.actor_id
        FROM event_log
        WHERE event_log.event_id IN (5,6,11,8,9,14,16,1,2,12,15,17)
    ) events
    right join user on user.id = events.actor_id
    group by user.id
""",
    db,
)

df.fillna(0, inplace=True)

df["date"] = df["date"].map(pd.to_datetime)
inactive_users = df[
    (df["edits"] + df["reviews"] + df["taxonomy"] == 0)
    & (df["description"] != "NULL")
    & (df["description"] != "")
    & (df["date"] < pd.to_datetime("2021-12-01"))
].copy()
inactive_users.sort_values(["date"], inplace=True)
inactive_users

compact_inactive_users = inactive_users[["serlo_id", "username"]]

/tmp/ipykernel_8762/1782927800.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""


In [17]:
subscribers = requests.get(
    mailchimp_api_url,
    params={
        "apikey": MAILCHIMP_API_KEY,
        "status": "subscribed",
        "fields": "total_items",
    },
)
total_subs = subscribers.json()["total_items"]
print(f"Anzahl der Newsletter-Subscriptions: {total_subs}")

Anzahl der Newsletter-Subscriptions: 8189


In [18]:
subscribers = []
for i in range(math.ceil(total_subs / 1000)):
    subscribers.extend(
        requests.get(
            mailchimp_api_url,
            params={
                "apikey": MAILCHIMP_API_KEY,
                "status": "subscribed",
                "since_timestamp_opt": date,
                "offset": (i * 1000),
                "count": 1000,
            },
        ).json()["members"]
    )

In [ ]:
has_not_opened_newsletter = list(
    filter(lambda subscriber: (subscriber["stats"]["avg_open_rate"] == 0), subscribers)
)

print(
    f"Anzahl Subscriber, die keinen Newsletter geöffnet haben: {len(has_not_opened_newsletter)}"
)

In [55]:
merged_data = {}

for subscriber in has_not_opened_newsletter:
    unique_key = subscriber["merge_fields"]["UNAME"]
    merged_data[unique_key] = subscriber

for index, user in compact_inactive_users.iterrows():
    #     print(user)
    unique_key = user["username"]
    if unique_key in merged_data:
        merged_data[unique_key].update(user)

linked_account = list(
    value
    for value in merged_data.values()
    if all(key in value for key in ("serlo_id", "id"))
)
df = pd.json_normalize(linked_account)
df.rename(columns={"id": "mailchimp_id"}, inplace=True)
# print(df["serlo_id", "username", "mailchimp_id", "email_address", "full_name",])
df

""


In [ ]:
"""
Das ist Code, den ich zum Testen verwende.

MY_TEST_MAILCHIMP_API_KEY = ""
dc2 = MY_TEST_MAILCHIMP_API_KEY.split('-')[1]
act_url = f"https://{dc2}.api.mailchimp.com/3.0/lists/66709f13c5/members"
activity = requests.get(act_url, params = {"apikey": MY_TEST_MAILCHIMP_API_KEY})
activity.json()
"""

In [ ]:
"""
members = activity.json()["members"]
test = filter(lambda member: (member["stats"]["avg_open_rate"] != 0) & (member["status"] == "subscribed"), members)
list(test)
"""

In [57]:
def is_possible_fake_account(subscriber):
    return subscriber["UNAME"] in [user["username"] for user in compact_inactive_users]

In [59]:
# filtered = filter(lambda subscriber: is_possible_fake_account(subscriber), linked_account)
filtered = filter(is_possible_fake_account(subscriber), linked_account)
print(
    "Anzahl Subscriber, die keinen Newsletter geöffnet haben und keine Aktivität auf serlo.org haben: "
    + len(filtered)
)
print("Ids: " + filtered.id)

KeyError: 'UNAME'

In [ ]:
### Das sind alles Subscriber, von denen wir auch den Account auf serlo.org kennen.
### You can check the profile of the users here: serlo.org/{id}
### Die Liste der Ids samt Usernamen, die von Serlo gelöscht werden sollen, bitte an moritz@serlo.org schicken oder,
### falls nicht verfügbar, an den Admin eures Vertrauens ;)
### Falls nur die Subscription gelöscht werden soll, bitte in den nächsten Abschnitt schauen.

In [60]:
# TODO: Which information is relevant?
not_linked_subscribers = list(
    filter(
        lambda member: not member["merge_fields"]["UNAME"], has_not_opened_newsletter
    )
)
print(
    pd.json_normalize(not_linked_subscribers)[
        ["id", "email_address", "full_name", "stats.timestamp_signup"]
    ]
)

KeyError: "['stats.timestamp_signup'] not in index"

In [ ]:
# Fill in here the mailchimp id (important: not the serlo id!) of the subscribers you want to remove from the newsletter list like that [id1, id2, id3, ...]
# TODO: Abklären, ob die komplett gelöscht werden sollen oder nur unsubscribed.
ids_to_remove = []

In [ ]:
delete_url(subscriber_hash):
    return f"/lists/a7bb2bbc4f/members/{subscriber_hash}/actions/delete-permanent"

# for subscriber_hash in ids_to_remove:
#     requests.post(delete_url(subscriber_hash), key: MAILCHIMP_API_KEY)